In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from DS_data_transformation import get_prm
from DS_Planck_Unet import val_pix

In [2]:
def gen_catalog(models, big_pix, cat_name, step=8, thr=0.1, save_inter_cats=None):
    from tqdm.notebook import tqdm
    
    from DS_detector import gen_pics_for_detection, detect_clusters_connected
    from DS_Planck_Unet import load_planck_model
    
    for model_name in tqdm(models):
        cur_cat = []
        for i in big_pix:
            model = load_planck_model(models[model_name])
            all_dict = gen_pics_for_detection(i, model, step=step)
            coords = detect_clusters_connected(all_dict, thr, i)
            cur_cat.append(coords)
            if not (save_inter_cats is None):
                coords.to_csv(save_inter_cats.format(pix=i, model=model_name), index=False)
        cur_cat = pd.concat(cur_cat, ignore_index=True)
        cur_cat.to_csv(cat_name.format(model=model_name, thr=thr, step=step), index=False)

In [3]:
models_dir = '/home/rt2122/Models/act_ros_rot/'
models_files = next(os.walk(models_dir))[-1]

In [4]:
cats_dir = '/home/rt2122/Data/detected_cats/act_ros_rot/'
detected_eps = next(os.walk(cats_dir))[-1]
detected_eps = [int(get_prm('rot', file)) for file in detected_eps]
repr(detected_eps)

'[3, 14, 22, 28, 31, 11, 35, 25, 12, 38, 10, 17, 45, 47, 21, 44, 15, 4, 26, 5, 24, 37, 27, 32, 41, 30, 43, 34, 46, 23, 42, 40, 36, 18, 1, 6, 33, 19, 16]'

In [5]:
models = {int(get_prm('ep', file)) : os.path.join(models_dir, file) for file in models_files 
          if not (int(get_prm('ep', file)) in detected_eps)}

In [6]:
len(models)

11

In [7]:
gen_catalog(models, val_pix, 
            '/home/rt2122/Data/detected_cats/act_ros_rot/val_pz_act_rot{model}_thr{thr}_step{step}.csv',
           save_inter_cats='/home/rt2122/Data/detected_cats/act_ros_rot/tmp/ep{model}_p{pix}.csv')

/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
